# KNN
'Dime con quien andas y te dire quien eres'

In [17]:
%matplotlib inline

import os

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm

import utils

sns.set(color_codes=True)
sns.set_context("notebook", font_scale=1)
plt.rcParams['figure.figsize'] = (17, 5)

In [18]:
import time
import os

import IPython.display as ipd
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd

from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, LabelBinarizer, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.multiclass import OneVsRestClassifier

In [19]:
tracks = pd.read_pickle("clean_data/track.pkl")
tracks.sample(3)

,date_created,duration,genre_top,title,album,album_tracks,artist,location,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
track_id,,,,,,,,,,,,,,,,
620,2008-11-26 02:07:58,205,Folk,Arabesque,Viridian City,13,Ed Askew,"New York, NY",0.885288,0.787973,0.236481,0.907792,0.117599,0.036578,140.057,0.477212
32909,2010-07-20 15:49:36,188,Electronic,La sauveur de l'humanité,Revolutionner la vie quotidienne,10,Jungle is neutral,"Cannes, France",0.735482,0.187130,0.313661,0.882947,0.093486,0.050955,131.507,0.038976
32470,2010-07-06 19:26:13,216,Hip-Hop,True,The Classic Material Completion Package XE,40,CM aka Creative,NaN,0.865402,0.760538,0.271308,0.067652,0.094511,0.136035,85.037,0.812242


In [20]:
features = pd.read_pickle("clean_data/features.pkl")
features.sample(3)

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
1924          1.846423 -0.145231  0.592124  0.729977  0.502047  0.388517   
21011         4.092313 -0.413204 -0.505495 -0.805989 -0.160786 -0.546243   
306           1.593105  2.611686  0.410375 -0.410679  0.054634  0.254629   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
track_id                                            ...                       
1924        1.108206  2.761793 -0.047858  0.670912  ...  0.082655  0.017663   
21011      -0.272130  0.569851 -0.591402 -0.896933  ...  0.090293  0.019892   
306         0.824009  0.855927 -1.322009 -1.121786  ...  0.062518  0.019641   

feature                     zcr                                          \
statistics             kurtosis       max      mean    median       min   
number            06         01        01        01        01        01   
track_id                                                                  
1924        0.020302  16.895546  0.480469  0.083016  0.081543  0.000000   
21011       0.015130  45.276386  0.709961  0.050770  0.035156  0.002441   
306         0.030675  29.590654  0.329102  0.028575  0.021484  0.000000   

feature                         
statistics      skew       std  
number            01        01  
track_id                        
1924        1.788843  0.028831  
21011       5.212354  0.057722  
306         4.548738  0.027111  

[3 rows x 518 columns]

In [21]:
features['mfcc']['std']

number,01,02,03,04,05,06,07,08,09,10,11,12,13,14,15,16,17,18,19,20
track_id,,,,,,,,,,,,,,,,,,,,
2,97.809044,38.569584,22.576273,20.767921,19.869740,20.299164,14.631481,12.183411,9.399827,10.742301,10.059609,8.601942,9.284250,9.245516,8.520863,8.560472,7.651871,7.246555,7.077188,7.391859
3,111.686371,41.191982,19.406340,22.025253,19.330267,19.175596,12.421652,10.261386,9.386757,10.165544,8.771966,10.026867,6.978541,7.650417,9.600357,7.222888,8.398293,7.285423,7.417791,8.777440
5,95.049904,39.367599,18.873550,24.319347,23.159674,17.159710,13.051704,10.914735,9.775167,11.808991,8.863638,9.581952,8.895723,8.141456,8.201844,7.780963,7.132692,7.539753,8.452527,7.334442
10,102.743553,44.409843,29.066904,14.013119,15.492754,16.689489,10.712612,11.680214,9.722768,11.369185,8.289734,7.985110,7.075400,6.972649,7.071393,7.270959,7.051070,6.928591,6.430473,6.186294
134,102.779747,47.266518,31.092169,36.841534,22.690844,22.462614,14.619110,13.056998,11.885354,12.076448,9.891955,9.696905,8.477474,7.413949,7.518894,6.755280,6.344675,6.761541,7.134986,6.803034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124718,82.199074,40.495678,28.313253,27.061916,23.061214,20.097122,15.845349,14.267061,12.304373,12.377727,11.354317,10.440834,8.641155,9.040728,8.051646,8.527964,8.388787,7.834747,8.285661,7.696751
124719,73.569328,47.098072,25.108372,23.497755,22.958052,17.311794,17.322132,13.303699,12.652544,11.973145,12.609986,12.356344,11.017785,10.632156,10.060791,10.510209,9.711658,8.551052,8.498206,8.861261
124720,76.106773,40.105381,23.174105,31.226048,21.046316,22.119991,16.210539,15.339581,11.790701,11.745691,11.238714,9.884409,9.637644,11.065310,8.631693,7.797875,7.775733,8.558174,7.195822,7.540391


In [22]:
genres = list(LabelEncoder().fit(tracks['genre_top']).classes_)
print('Top genres ({}): {}'.format(len(genres), genres))

Top genres (12): ['Blues', 'Classical', 'Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Jazz', 'Old-Time / Historic', 'Pop', 'Rock']


In [23]:
x = features[features.index.isin(tracks.index)]
y = tracks['genre_top']

## Entrenamiento con mfcc
MFCC: Mel Frequency Cepstral Coefficient
(MFCC), used for for identifying the linguistic content and discarding all the other stuff which carries information like background noise, emotion etc. Usually this related to the musical instrument.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x['mfcc'], y, random_state=0)
print('Hay', y_train.shape[0], 'datos de entrenamiento y', y_test.shape[0], 'datos de testeo')

## Pre-process

In [27]:
from sklearn.model_selection import train_test_split

def pre_process(fset):
    enc = LabelEncoder()

    X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=0)

    y_train = enc.fit_transform(y_train)
    y_test = enc.transform(y_test)

    X_train = X_train.loc[fset].as_matrix()
    X_test = X_test.loc[fset].as_matrix()

    X_train, y_train = shuffle(X_train, y_train, random_state=42)

    # Standardize features by removing the mean and scaling to unit variance.
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)

    print('Hay', y_train.shape[0], 'datos de entrenamiento y', y_test.shape[0], 'datos de testeo')
    return X_train, X_test, y_train, y_test

In [28]:
def test_classifiers_features(classifiers, feature_sets, multi_label=False):
    columns = list(classifiers.keys()).insert(0, 'dim')
    scores = pd.DataFrame(columns=columns, index=feature_sets.keys())
    times = pd.DataFrame(columns=classifiers.keys(), index=feature_sets.keys())
    for fset_name, fset in tqdm_notebook(feature_sets.items(), desc='features'):
        X_train, X_test, y_train, y_test = pre_process(fset)
        scores.loc[fset_name, 'dim'] = X_train.shape[1]
        for clf_name, clf in classifiers.items():
            t = time.process_time()
            clf.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            scores.loc[fset_name, clf_name] = score
            times.loc[fset_name, clf_name] = time.process_time() - t
    return scores, times


In [29]:
def format_scores(scores):
    def highlight(s):
        is_max = s == max(s[1:])
        return ['background-color: yellow' if v else '' for v in is_max]
    scores = scores.style.apply(highlight, axis=1)
    return scores.format('{:.2%}', subset=pd.IndexSlice[:, scores.columns[1]:])

In [34]:
classifiers = {
    'LR': LogisticRegression(),
    'kNN': KNeighborsClassifier(n_neighbors=200),
    'SVCrbf': SVC(kernel='rbf'),
    'SVCpoly1': SVC(kernel='poly', degree=1),
    'linSVC1': SVC(kernel="linear"),
    'linSVC2': LinearSVC(),
    #GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True),
    'DT': DecisionTreeClassifier(max_depth=5),
    'RF': RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    'AdaBoost': AdaBoostClassifier(n_estimators=10),
    'MLP1': MLPClassifier(hidden_layer_sizes=(100,), max_iter=2000),
    'MLP2': MLPClassifier(hidden_layer_sizes=(200, 50), max_iter=2000),
    'NB': GaussianNB(),
    'QDA': QuadraticDiscriminantAnalysis(),
}

feature_sets = {
#    'echonest_audio': ('echonest', 'audio_features'),
#    'echonest_social': ('echonest', 'social_features'),
#    'echonest_temporal': ('echonest', 'temporal_features'),
#    'echonest_audio/social': ('echonest', ('audio_features', 'social_features')),
#    'echonest_all': ('echonest', ('audio_features', 'social_features', 'temporal_features')),
}
for name in features.columns.levels[0]:
    feature_sets[name] = name
    
feature_sets.update({
    'mfcc/contrast': ['mfcc', 'spectral_contrast'],
    'mfcc/contrast/chroma': ['mfcc', 'spectral_contrast', 'chroma_cens'],
    'mfcc/contrast/centroid': ['mfcc', 'spectral_contrast', 'spectral_centroid'],
    'mfcc/contrast/chroma/centroid': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid'],
    'mfcc/contrast/chroma/centroid/tonnetz': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'tonnetz'],
    'mfcc/contrast/chroma/centroid/zcr': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'zcr'],
    'all_non-echonest': list(features.columns.levels[0])
})

In [35]:
scores, times = test_classifiers_features(classifiers, feature_sets)

#ipd.display(format_scores(scores))
#ipd.display(times.style.format('{:.4f}'))

/Users/user/.pyenv/versions/anaconda3-5.3.1/envs/dhdsblend/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


KeyError: 'chroma_cens'